# Migration experience premium (Chapter 2) - TABLES

In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('D:\KGZ_LiK\Chapter_2')
os.getcwd()

'D:\\KGZ_LiK\\Chapter_2'

In [2]:
### Connect to Stata
import sys
sys.path.append('D:/Moved_from_C/Stata17/utilities')
from pystata import config
config.init('mp');


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: 微软用户
               微软中国

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


# 1. Summary statistics with t-test by migration experience

In [5]:
%%stata
use data_2.dta, clear

keep if oilrentpercap19!=.

winsor imputed_ldwage, p(.01) gen(ldw)
winsor imputed_ldwagew, p(.01) gen(ldwh)

global all age yob edc* married dependents urban ///
			mp bsk south exp employed selfemp ///
			agriculture social transport construction ///
			ldwage ldw ldwagew ldwh ///
			oilrentpercap19

## Table 1
sum $all


. use data_2.dta, clear

. 
. keep if oilrentpercap19!=.
(6,443 observations deleted)

. 
. winsor imputed_ldwage, p(.01) gen(ldw)

. winsor imputed_ldwagew, p(.01) gen(ldwh)

. 
. global all age yob edc* married dependents urban ///
>                         mp bsk south exp employed selfemp ///
>                         agriculture social transport construction ///
>                         ldwage ldw ldwagew ldwh ///
>                         oilrentpercap19

. 
. ## Table 1
Unknown #command
. sum $all

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
         age |     13,293    28.35417    7.712847         16         48
         yob |     13,293     1985.18    7.520358       1971       2002
        edc1 |     13,293    .1255548    .3313594          0          1
        edc2 |     13,293    .6291281    .4830564          0          1
        edc3 |     13,293    .0983977    .2978628      

In [7]:
%%stata
# Table 1. Summary statistics with t-test
table1, by(mp) vars(age contn\                                ///
        dependents contn\ married contn\                      ///
        urban contn\ south contn\ bsk contn\                  ///
        educc cat\                                            ///
        employed contn\ selfemp contn\ hours contn\           ///
        ldwage contn\ ldw contn\ ldwagew contn\	ldwh contn\   ///
        agriculture contn\ social contn\                      ///
        construction contn\ transport contn\)                 ///
        test onecol format(%9.2fc) cformat(%9.2fc)            ///
        saving(table1.xlsx, replace)


. # Table 1. Summary statistics with t-test
Unknown #command
. table1, by(mp) vars(age contn\                                ///
>         dependents contn\ married contn\                      ///
>         urban contn\ south contn\ bsk contn\                  ///
>         educc cat\                                            ///
>         employed contn\ selfemp contn\ hours contn\           ///
>         ldwage contn\ ldw contn\ ldwagew contn\ ldwh contn\   ///
>         agriculture contn\ social contn\                      ///
>         construction contn\ transport contn\)                 ///
>         test onecol format(%9.2fc) cformat(%9.2fc)            ///
>         saving(table1.xlsx, replace)
  +----------------------------------------------------------------------------
> -----------------------------------------------------------------+
  | Factor                                                                     
>  mp = 0          mp = 1          p-value   Test         

# 2. Estimates for effect of migration expereince on employment
## Table 2 and 2A

In [3]:
%%stata 

use data_2.dta, clear

keep if oilrentpercap19!=.
keep age exp exp2 urban dependents edc* rgc* yrs* skill employed oilrentpercap19 mp

global w exp exp2 urban dependents edc* rgc* yrs*
global e employed
global se cluster skill
global m oilrentpercap19

eststo clear
#drop mpwm
qui probit $e mp $w, vce($se)
eststo

qui probit mp $m $w, vce($se)
predict mpwm
eststo
qui reg mp mpwm $w, vce($se)
eststo
qui ivregress 2sls $e $w (mp = mpwm), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

qui reg mp $m $w, vce($se)
eststo
qui ivregress 2sls $e $w (mp = $m), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

esttab, r2 pr2 b(4) se(5) scalar(F1) sfmt(4) keep(mp* $m mpwm)
#esttab using table2.rtf, r2 pr2 b(4) se(4) scalar(F1) sfmt(4) keep(mp* $m mpwm)
#esttab using table2.rtf, r2 pr2 b(4) se(4) scalar(F1) sfmt(4) drop(yrs* rgc*) ##This is a full table for Table 2A


. 
. use data_2.dta, clear

. 
. keep if oilrentpercap19!=.
(6,443 observations deleted)

. keep age exp exp2 urban dependents edc* rgc* yrs* skill employed oilrentperca
> p19 mp

. 
. global w exp exp2 urban dependents edc* rgc* yrs*

. global e employed

. global se cluster skill

. global m oilrentpercap19

. 
. eststo clear

. #drop mpwm
Unknown #command
. qui probit $e mp $w, vce($se)

. eststo
(est1 stored)

. 
. qui probit mp $m $w, vce($se)

. predict mpwm
(option pr assumed; Pr(mp))

. eststo
(est2 stored)

. qui reg mp mpwm $w, vce($se)

. eststo
(est3 stored)

. qui ivregress 2sls $e $w (mp = mpwm), vce($se)

. eststo
(est4 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = fstat[1,4]

. 
. qui reg mp $m $w, vce($se)

. eststo
(est5 stored)

. qui ivregress 2sls $e $w (mp = $m), vce($se)

. eststo
(est6 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = fstat[1,4]

. 
. esttab, r2 pr2 b(4) se(5) s

# 3. Wage effects of migration experience
## Table 3 and 3A

In [4]:
%%stata
use data_2.dta, clear

winsor imputed_ldwage, p(.01) gen(ldw)
keep if oilrentpercap19!=. & ldw!=. & hours!=.

global w exp exp2 hours i.educc urban rgc* yrs* 
global se cluster skill
global wage ldw
global m oilrentpercap19

eststo clear

qui reg $wage mp $w, vce($se)
eststo

qui probit mp $m $w if ldw!=., vce($se)
predict mpwm
eststo
qui reg mp mpwm $w if ldw!=., vce($se)
eststo
qui ivregress 2sls $wage $w (mp = mpwm), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

qui reg mp $m $w if ldw!=., vce($se)
eststo
qui ivregress 2sls $wage $w (mp = $m), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

esttab, b se ar2 pr2 scalar(F1) keep(mp* $m mpwm)
#esttab using table3.rtf, r2 pr2 b(4) se(5) scalar(F1) sfmt(4) drop(yrs* rgc*) # For Table 3A 


. use data_2.dta, clear

. 
. winsor imputed_ldwage, p(.01) gen(ldw)

. keep if oilrentpercap19!=. & ldw!=. & hours!=.
(12,614 observations deleted)

. 
. global w exp exp2 hours i.educc urban rgc* yrs* 

. global se cluster skill

. global wage ldw

. global m oilrentpercap19

. 
. eststo clear

. 
. qui reg $wage mp $w, vce($se)

. eststo
(est1 stored)

. 
. qui probit mp $m $w if ldw!=., vce($se)

. predict mpwm
(option pr assumed; Pr(mp))

. eststo
(est2 stored)

. qui reg mp mpwm $w if ldw!=., vce($se)

. eststo
(est3 stored)

. qui ivregress 2sls $wage $w (mp = mpwm), vce($se)

. eststo
(est4 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = fstat[1,4]

. 
. qui reg mp $m $w if ldw!=., vce($se)

. eststo
(est5 stored)

. qui ivregress 2sls $wage $w (mp = $m), vce($se)

. eststo
(est6 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = fstat[1,4]

. 
. esttab, b se ar2 pr2 scalar(F1) keep(mp* $m mpwm)



# 4. Wage effects of migration experience controlling sector and occupation
## Table 4 and 4A

In [5]:
%%stata
use data_2.dta, clear

winsor imputed_ldwage, p(.01) gen(ldw)
keep if oilrentpercap19!=. & ldw!=. & hours!=.

global w exp exp2 hours i.educc urban rgc* yrs* 
global se cluster skill
global wage ldw
global m oilrentpercap19
global s i.sector i.occupation

eststo clear
qui reg $wage mp $w $f $s, vce($se)
eststo

qui probit mp $m $w $f $s if ldw!=., vce($se)
predict mpwm
eststo
qui reg mp mpwm $w $f $s if ldw!=., vce($se)
eststo
qui ivregress 2sls $wage $w $f $s (mp = mpwm), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

qui reg mp $m $w $f $s if ldw!=., vce($se)
eststo
qui ivregress 2sls $wage $w $f $s (mp = $m), vce($se)
eststo
qui estat firststage
mat fstat = r(singleresults)
qui estadd scalar F1 = fstat[1,4]

esttab, b se ar2 pr2 scalar(F1) keep(mp* $m mpwm)
#esttab using table4.rtf, r2 pr2 b(4) se(5) scalar(F1) sfmt(4) drop(yrs* rgc* sector* occupation*) replace # For Table4a


. use data_2.dta, clear

. 
. winsor imputed_ldwage, p(.01) gen(ldw)

. keep if oilrentpercap19!=. & ldw!=. & hours!=.
(12,614 observations deleted)

. 
. global w exp exp2 hours i.educc urban rgc* yrs* 

. global se cluster skill

. global wage ldw

. global m oilrentpercap19

. global s i.sector i.occupation

. 
. eststo clear

. qui reg $wage mp $w $f $s, vce($se)

. eststo
(est1 stored)

. 
. qui probit mp $m $w $f $s if ldw!=., vce($se)

. predict mpwm
(option pr assumed; Pr(mp))

. eststo
(est2 stored)

. qui reg mp mpwm $w $f $s if ldw!=., vce($se)

. eststo
(est3 stored)

. qui ivregress 2sls $wage $w $f $s (mp = mpwm), vce($se)

. eststo
(est4 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = fstat[1,4]

. 
. qui reg mp $m $w $f $s if ldw!=., vce($se)

. eststo
(est5 stored)

. qui ivregress 2sls $wage $w $f $s (mp = $m), vce($se)

. eststo
(est6 stored)

. qui estat firststage

. mat fstat = r(singleresults)

. qui estadd scalar F1 = f

# 5. Wage effects of migration experience by education level
## Table 5 and 5A

In [8]:
%%stata
use data_2.dta, clear

winsor imputed_ldwage, p(.01) gen(ldw)
keep if oilrentpercap19!=. & ldw!=. & hours!=.

local edulist edc1 edc2 edc3 edc4

global se cluster skill
global wage ldw
global m oilrentpercap19
global w exp exp2 hours urban rgc* yrs*
#global w exp exp2 hours urban 


eststo clear

foreach num of numlist 1/4 {
	qui probit mp $m $w $f if edc`num' == 1 & ldw!=., vce($se)
	predict mpwm
	qui ivregress 2sls $wage $w $f (mp = mpwm) if edc`num' == 1 & ldw!=1, vce($se)
	eststo Model`num'
	qui estat firststage
	mat fstat = r(singleresults)
	qui estadd scalar F1 = fstat[1,4]
	drop mpwm
 	}
esttab, b(4) se(4) ar2 scalar(F1) keep(mp)
#esttab using table5a.rtf, ar2 b(4) se(4) scalar(F1) keep(exp* hours urban) replace ## For Table 5A of Figure 6 in appendix


. use data_2.dta, clear

. 
. winsor imputed_ldwage, p(.01) gen(ldw)

. keep if oilrentpercap19!=. & ldw!=. & hours!=.
(12,614 observations deleted)

. 
. local edulist edc1 edc2 edc3 edc4

. 
. global se cluster skill

. global wage ldw

. global m oilrentpercap19

. global w exp exp2 hours urban rgc* yrs*

. #global w exp exp2 hours urban 
Unknown #command
. 
. 
. eststo clear

. 
. foreach num of numlist 1/4 {
  2.         qui probit mp $m $w $f if edc`num' == 1 & ldw!=., vce($se)
  3.         predict mpwm
  4.         qui ivregress 2sls $wage $w $f (mp = mpwm) if edc`num' == 1 & ldw!
> =1, vce($se)
  5.         eststo Model`num'
  6.         qui estat firststage
  7.         mat fstat = r(singleresults)
  8.         qui estadd scalar F1 = fstat[1,4]
  9.         drop mpwm
 10.         }
(option pr assumed; Pr(mp))
(946 missing values generated)
(option pr assumed; Pr(mp))
(option pr assumed; Pr(mp))
(option pr assumed; Pr(mp))

. esttab, b(4) se(4) ar2 scalar(F1) keep(mp)

-------

# Done